# download & import libraries

In [28]:
!pip install git+https://github.com/huggingface/transformers.git@main accelerate


  Cloning https://github.com/huggingface/transformers.git (to revision main) to /tmp/pip-req-build-5kleoaef
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-5kleoaef
  Resolved https://github.com/huggingface/transformers.git to commit 4edffda636fb2bf673282b31163e598b5872994e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [1]:
pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 46.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 14.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.8/304.8 kB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 64.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 10.2 M

In [2]:
%%capture
!pip install ultralytics

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
%%capture
!unzip  /content/drive/MyDrive/cp_proj/CarDD_release.zip

In [5]:
%%capture

!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

##load models and define necessary functions

In [1]:
import os
import tensorflow as tf
import cv2
from tensorflow import keras
import numpy as np
from ultralytics import YOLO
import  json
import torch
from matplotlib import pyplot as plt
from PIL import Image

In [2]:
import logging
from collections import OrderedDict
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries


# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.engine import DefaultTrainer
from detectron2.utils.visualizer import ColorMode
from detectron2.solver import build_lr_scheduler, build_optimizer
from detectron2.checkpoint import DetectionCheckpointer, PeriodicCheckpointer
from detectron2.utils.events import EventStorage
from detectron2.modeling import build_model
import detectron2.utils.comm as comm
from detectron2.engine import default_argument_parser, default_setup, default_writers, launch


import gradio as gr
import transformers

from transformers import AutoTokenizer
import torch

from google.colab.patches import cv2_imshow


In [3]:
register_coco_instances("coco_dataset", {},
                            "/content/CarDD_release/CarDD_COCO/annotations/instances_val2017.json",
                            "/content/CarDD_release/CarDD_COCO/val2017")

In [4]:
Ai_image_detector =tf.keras.models.load_model('/content/drive/MyDrive/cp_proj/AI_Or_NOT.h5')
damage_classifier =tf.keras.models.load_model('/content/drive/MyDrive/cp_proj/damage_classifier.h5')
car_segmentation =YOLO('/content/drive/MyDrive/cp_proj/Car_parts.pt')
damage_seg_path='/content/drive/MyDrive/cp_proj/damageseg.pth'

In [5]:
def prepare(filepath,im_size):
  IMG_SIZE = im_size
  img_array = cv2.imread(filepath)
  img_array = cv2.cvtColor(img_array ,cv2.COLOR_BGR2RGB)
  new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
  return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 3)

In [6]:
def Ai_detector( photo_path, model=Ai_image_detector,im_size=350):

  prediction =model.predict(prepare(photo_path,im_size))
  if prediction[0][0]>=0.5:
    return 0 #real image
  else:
    return 1 #AI_generated


In [7]:
def damage_classification( photo_path, model=damage_classifier,im_size=450):
  class_labels={0:'minor',1: 'moderate',2: 'severe'}

  prediction =model.predict(prepare(photo_path,im_size))
  class_index = np.argmax(prediction)
  return class_labels[class_index]

In [8]:
def prepare_for_segmentation(filepath,im_size=640):
  IMG_SIZE = im_size
  img_array = cv2.imread(filepath)
  img_array = cv2.cvtColor(img_array ,cv2.COLOR_BGR2RGB)
  new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
  return new_array

In [9]:
damage_categories= [ "dent","scratch","crack","glass shatter","lamp broken","tire flat"]
parts_categories=['Bonnet', 'Frontbumper', 'Frontdoor', 'Frontfender', 'Headlights', 'Rearbumper', 'Reardoor', 'Rearfender', 'Rearlamp', 'Rockerpanel', 'Sidemirror', 'Trunklid', 'Wheel', 'Windshield']

In [10]:
class damage_mask:
  def __init__(self,model_path,number_of_classes):
    # Assuming your dataset is in COCO format


    # Getting the metadata
    self.car_damage_metadata = MetadataCatalog.get("coco_dataset")
    self.weights_path=model_path
    self.number_of_classes=number_of_classes
  def setup_cfg(self):
      """
      Set up and return the Detectron2 config.
      """
      cfg = get_cfg()
      cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
      cfg.MODEL.ROI_HEADS.NUM_CLASSES = self.number_of_classes # Change as per your model's classes
      cfg.MODEL.WEIGHTS = self.weights_path
      cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # Set the detection threshold
      cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
      return cfg

  def load_model(self):
      """
      Load the Detectron2 model with the specified weights.
      """
      cfg = self.setup_cfg()
      return DefaultPredictor(cfg)


  def detect_damage(self,image):
      """
      Perform car damage detection and return the image with visualized predictions.
      """
      model=self.load_model()
      outputs = model(image)
      v = Visualizer(image[:, :, ::-1],
                    metadata=self.car_damage_metadata,
                    scale=1,)
      out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
      print(outputs["instances"].to("cpu"))
      return outputs["instances"].to("cpu") , Image.fromarray(out.get_image()[:, :, ::-1])


In [11]:


def process_damage_info(results, preds, parts_categories, damage_categories, image_path='/content/cardent.jpg'):
    car_parts_classes = [int(x) for x in results[0].boxes.cls.tolist()]
    car_parts_bboxes = results[0].boxes.xyxy.tolist()
    car_parts_seg_masks = results[0].masks.data.tolist()
    damage_classes = preds.pred_classes.tolist()
    damage_bbox = preds.pred_boxes.tensor.tolist()
    damage_segmentation_masks = preds.pred_masks.tolist()

    damage_info_list = []
    image = cv2.imread(image_path)
    image = cv2.resize(image, (640, 640))

    for part_index, part_class in enumerate(car_parts_classes):
        if part_index < len(damage_classes):
            damage_class = damage_classes[part_index]
            damage_bbox_part = car_parts_bboxes[part_index]
            damage_mask_part = damage_segmentation_masks[part_index]

            intersection = np.logical_and(car_parts_seg_masks[part_index], damage_mask_part)
            iou = np.sum(intersection) / np.sum(car_parts_seg_masks[part_index])

            x, y, w, h = map(int, damage_bbox_part)
            cv2.rectangle(image, (x, y), (w, h), (0, 255, 0), 2)

            damage_mask = np.array(damage_mask_part, dtype=np.uint8) * 255
            contours, _ = cv2.findContours(damage_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            cv2.drawContours(image, contours, -1, (0, 0, 255), 2)
            text = f"Part: {parts_categories[part_class]}, Damage: {damage_categories[damage_class]}, Percentage: {iou * 100:.2f}%"
            cv2.putText(image, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

            # Build the damage information string
            damage_info_str = f"{damage_categories[damage_class]} detected in {parts_categories[part_class]} with severity {iou * 100:.2f}%"
            damage_info_list.append(damage_info_str)

    return damage_info_list, image




# load llama model for price estimation

In [13]:
model = "codellama/CodeLlama-7b-Instruct-hf"

In [14]:
tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,

    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# Framework Assembled :)

In [18]:
def process_image(image_path):
    if Ai_detector(image_path, Ai_image_detector) == 1:
        return  cv2.imread(image_path), 'This is an AI-generated image'
    else:
        return process_non_generated_image(image_path)

def process_non_generated_image(image_path):
    Accident_severity = damage_classification(image_path, damage_classifier)
    seg_img = prepare_for_segmentation(image_path)
    results = car_segmentation(seg_img)
    damage = damage_mask(damage_seg_path, 6)

    preds, image = damage.detect_damage(seg_img)
    damage_info_list, image = process_damage_info(results, preds, parts_categories, damage_categories, image_path=image_path)
    description = f"The car has suffered {Accident_severity} damage. Here are the details of the damaged parts: {damage_info_list}"
    system_prompt ='You are an assistant with expertise in car repairs. Your goal is to provide accurate and concise estimates of the repair cost  based solely on the provided information.'
    B_INST, E_INST = "[INST]", "[/INST]"
    B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
    user_prompt=f"I've been in a car accident and my car has been damaged. Here's what happened:{description} . As a mechanic,  based solely on the provided information., could you provide an estimate for the repair costs?"
    prompt = f"{B_INST} {B_SYS}{system_prompt.strip()}{E_SYS}{user_prompt.strip()} {E_INST}"
    sequences = pipeline(
    prompt,
    do_sample=True,
    top_k=10,
    temperature=0.1,
    top_p=0.95,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=350,
    add_special_tokens=False
)

    return image ,sequences[0]['generated_text']

iface = gr.Interface(
    fn=process_image,
    inputs=gr.Image(type="filepath", label="Upload Image"),
    outputs=["image","textbox"]
)



iface.launch(debug=True)


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://7a52dfc976aa293302.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


1/1 [==============================] - 0s 72ms/step

0: 640x640 1 Frontdoor, 230.5ms
Speed: 3.2ms preprocess, 230.5ms inference, 7.2ms postprocess per image at shape (1, 3, 640, 640)
[12/19 02:18:26 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /content/drive/MyDrive/cp_proj/damageseg.pth ...


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Instances(num_instances=1, image_height=640, image_width=640, fields=[pred_boxes: Boxes(tensor([[147.5317,  95.6520, 526.0703, 509.3194]])), scores: tensor([0.9730]), pred_classes: tensor([0]), pred_masks: tensor([[[False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         ...,
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False]]])])
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://7a52dfc976aa293302.gradio.live
